<a href="https://colab.research.google.com/github/palominojulio/5_ways_2D_histograms/blob/master/2025_05_04_21_6_37_ATC_ESFERA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🧠 PASO 1: Cargar y preparar entorno en Google Colab

Subir tu estudio DICOM (como .zip).
1. Convertirlo a .nii.gz.
2. Crear una máscara esférica de 15 mm centrada en coordenadas que nos indiques.
3. Exportar esa máscara lista para usar en 3D Slicer o PyRadiomics.

In [ ]:
# ✅ INSTALAR LIBRERÍAS NECESARIAS

!pip install -q dicom2nifti nibabel pydicom

import os
import dicom2nifti
import nibabel as nib
import numpy as np
from google.colab import files

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 93.5 MB/s eta 0:00:00


🧠 PASO 1: Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Parte fija del path
base_dir = "/content/drive/MyDrive/0.UIA.PACS/"

# Pedir solo el nombre de la subcarpeta DICOM
folder_name = input("📁 Ingresá el nombre de la carpeta DICOM dentro de 0.UIA.PACS:\n")

# Ruta completa al DICOM
dicom_dir = os.path.join(base_dir, folder_name)
nifti_output_dir = "/content"  # Salida en local


📁 Ingresá el nombre de la carpeta DICOM dentro de 0.UIA.PACS:
0.ATC.36519851216B182MB


✅ PASO 2: Solicita la ruta a la carpeta DICOM

✅ PASO 3: Convertir DICOM a NIfTI

In [ ]:
!pip install -q dicom2nifti nibabel pydicom

import dicom2nifti
import os

# Conversión
dicom2nifti.convert_directory(dicom_dir, nifti_output_dir, compression=True)

# Mostrar resultado
for f in os.listdir(nifti_output_dir):
    if f.endswith(".nii.gz"):
        nifti_path = os.path.join(nifti_output_dir, f)
        print("✅ NIfTI generado:", nifti_path)

✅ NIfTI generado: /content/9.nii.gz


✅ Se genera un volumen binario (.nii.gz) que contiene una esfera de radio 15 mm, centrada en la coordenada que corresponde al isocentro del cuello del aneurisma. Visto en 3D Slicer

✅ Se solicita al usuario las coordenasas del centro de la esfera.
✅ Crear un volumen esférico centrado en la coordenada
✅ La guarda como esfera_15mm.nii.gz en /content
✅ La renombra usando el mismo nombre de la carpeta DICOM original + E15mm.nii.gz
✅ La sube a tu Google Drive, dentro de 0.UIA.PACS/
✅ Copia ambos archivos a tu Google Drive en 0.UIA.PACS/.

In [ ]:
!pip install -q nibabel numpy

import nibabel as nib
import numpy as np
import os

# Ruta del NIfTI original
nifti_path = "/content/9.nii.gz"
img = nib.load(nifti_path)
data = img.get_fdata()
affine = img.affine

# Centro en voxel space
center_voxel = np.array([240, 193, 124])
radius_mm = 15

# Espaciado (mm por voxel)
spacing = img.header.get_zooms()

# Radio en voxels
radius_vox = np.round(radius_mm / np.array(spacing)).astype(int)

# Crear máscara esférica
zz, yy, xx = np.ogrid[:data.shape[2], :data.shape[1], :data.shape[0]]
dist = ((xx - center_voxel[0])**2 / radius_vox[0]**2 +
        (yy - center_voxel[1])**2 / radius_vox[1]**2 +
        (zz - center_voxel[2])**2 / radius_vox[2]**2)

mask = dist <= 1.0

# Aplicar máscara
subvolume = np.zeros_like(data)
subvolume[mask.T] = data[mask.T]

# Guardar nuevo NIfTI
output_name = "0.ATC.36519851216B182MB_E15mm.nii.gz"
output_path = os.path.join("/content", output_name)
nib.save(nib.Nifti1Image(subvolume, affine), output_path)

print(f"✅ Subvolumen esférico guardado en: {output_path}")




✅ Subvolumen esférico guardado en: /content/0.ATC.36519851216B182MB_E15mm.nii.gz


# ✅ PASO EXTRA: Copiar el archivo .nii.gz a Google Drive en 0.UIA.PACS

In [ ]:
# ✅ PASO EXTRA: Copiar el archivo .nii.gz a Google Drive en 0.UIA.PACS

import shutil

# Ruta destino: misma carpeta donde estaba el DICOM
destination_dir = os.path.join(base_dir)

# Comprobamos que existe el archivo y lo movemos
if os.path.exists(nifti_path):
    output_filename = f"{folder_name}_E15mm.nii.gz"
    output_path = os.path.join(destination_dir, output_filename)
    shutil.copy(nifti_path, output_path)
    print(f"✅ Archivo copiado a Drive como: {output_path}")
else:
    print("❌ Archivo NIfTI no encontrado para copiar.")


✅ Archivo copiado a Drive como: /content/drive/MyDrive/0.UIA.PACS/0.ATC.36519851216B182MB_E15mm.nii.gz


Corrección 20250504 23:11 [  ]

Te pide:

📁 Carpeta DICOM (0.UIA.PACS/xxxx)
📌 Coordenadas del centro (x,y,z)
🎯 Radio de la esfera (en mm)

1. Convierte DICOM a NIfTI
2. Genera una esfera con el radio que indiques
3. Guarda el volumen recortado como .nii.gz y .nrrd
4. Copia ambos archivos a Google Drive

In [1]:
# ─────────────────────────────────────
# 1. MONTAJE DE DRIVE Y LIBRERÍAS
# ─────────────────────────────────────
import os
import shutil
import numpy as np
import nibabel as nib
from google.colab import drive

!pip install -q dicom2nifti nibabel pydicom
!pip install -q pynrrd
import dicom2nifti
import nrrd

drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 104.1 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
# ─────────────────────────────────────
# 2. INPUTS DEL USUARIO
# ─────────────────────────────────────
base_dir = "/content/drive/MyDrive/0.UIA.PACS/"
folder_name = input("📁 Ingresá el nombre de la carpeta DICOM dentro de 0.UIA.PACS:\n")
coords_str = input("📍 Ingresá las coordenadas del centro de la esfera (formato: x,y,z):\n")
radius_mm = float(input("🟢 Ingresá el radio de la esfera en mm:\n"))

center_voxel = np.array([int(c) for c in coords_str.split(",")])
dicom_dir = os.path.join(base_dir, folder_name)
nifti_output_dir = "/content"

# ─────────────────────────────────────
# 3. CONVERSIÓN DICOM → NIFTI
# ─────────────────────────────────────
dicom2nifti.convert_directory(dicom_dir, nifti_output_dir, compression=True)

for f in os.listdir(nifti_output_dir):
    if f.endswith(".nii.gz"):
        nifti_path = os.path.join(nifti_output_dir, f)
        print("✅ NIfTI generado:", nifti_path)

# ─────────────────────────────────────
# 4. RECORTE DEL SUBVOLUMEN ESFÉRICO
# ─────────────────────────────────────
img = nib.load(nifti_path)
data = img.get_fdata()
affine = img.affine
spacing = img.header.get_zooms()

# Cálculo del radio en voxels
radius_vox = np.round(radius_mm / np.array(spacing)).astype(int)

# Crear máscara esférica en orden z, y, x
zz, yy, xx = np.ogrid[:data.shape[2], :data.shape[1], :data.shape[0]]
dist = ((xx - center_voxel[0])**2 / radius_vox[0]**2 +
        (yy - center_voxel[1])**2 / radius_vox[1]**2 +
        (zz - center_voxel[2])**2 / radius_vox[2]**2)
mask = dist <= 1.0
mask = mask.transpose(2, 1, 0)  # → a (x, y, z)

# Aplicar máscara
subvolume = np.zeros_like(data)
subvolume[mask] = data[mask]

# ─────────────────────────────────────
# 5. GUARDADO DE RESULTADOS
# ─────────────────────────────────────
output_base = f"{folder_name}_E{int(radius_mm)}mm"
nii_path = os.path.join(nifti_output_dir, output_base + ".nii.gz")
nib.save(nib.Nifti1Image(subvolume, affine), nii_path)
print(f"✅ Subvolumen guardado como NIfTI: {nii_path}")

# Guardar como NRRD (opcional para 3D Slicer)
nrrd_path = os.path.join(nifti_output_dir, output_base + ".nrrd")
header = {'space directions': np.diag(spacing), 'space origin': affine[:3, 3]}
nrrd.write(nrrd_path, subvolume, header)
print(f"🟢 Subvolumen guardado como NRRD: {nrrd_path}")

# ─────────────────────────────────────
# 6. COPIAR ARCHIVOS A GOOGLE DRIVE
# ─────────────────────────────────────
shutil.copy(nii_path, os.path.join(base_dir, os.path.basename(nii_path)))
shutil.copy(nrrd_path, os.path.join(base_dir, os.path.basename(nrrd_path)))
print("📁✅ Archivos copiados a Google Drive.")


📁 Ingresá el nombre de la carpeta DICOM dentro de 0.UIA.PACS:
0.ATC.36519851216B182MB
📍 Ingresá las coordenadas del centro de la esfera (formato: x,y,z):
240,193,240
🟢 Ingresá el radio de la esfera en mm:
30


✅ NIfTI generado: /content/9.nii.gz
✅ Subvolumen guardado como NIfTI: /content/0.ATC.36519851216B182MB_E30mm.nii.gz
🟢 Subvolumen guardado como NRRD: /content/0.ATC.36519851216B182MB_E30mm.nrrd
📁✅ Archivos copiados a Google Drive.


20250505 2:37 [Correccion]

✅ Conversión DICOM → NIfTI
✅ Recorte esférico centrado en coordenadas RAS
✅ Exportación en .nii.gz (con sform/qform)
✅ Exportación en .nrrd (LPS explícito)
✅ Fiducial .fcsv para verificar en 3D Slicer
✅ Segmentación .seg.nrrd directamente compatible con Segment Editor
✅ Copia automática de todo a tu Google Drive

In [ ]:
# ──────────────────────────────
# 1. MONTAJE Y LIBRERÍAS
# ──────────────────────────────
import os
import shutil
import numpy as np
import nibabel as nib
import dicom2nifti
import nrrd
from google.colab import drive

!pip install -q dicom2nifti nibabel pynrrd pydicom

drive.mount('/content/drive')

# ──────────────────────────────
# 2. INPUT DEL USUARIO
# ──────────────────────────────
base_dir = "/content/drive/MyDrive/0.UIA.PACS/"
folder_name = input("📁 Nombre de la carpeta DICOM dentro de 0.UIA.PACS:\n")
coords_str = input("📍 Coordenadas RAS desde 3D Slicer (formato: x,y,z):\n")
radius_mm = float(input("🟢 Radio de la esfera en mm:\n"))

center_ras = np.array([float(c) for c in coords_str.split(",")])
dicom_dir = os.path.join(base_dir, folder_name)
nifti_output_dir = "/content"

# ──────────────────────────────
# 3. CONVERSIÓN DICOM → NIFTI
# ──────────────────────────────
dicom2nifti.convert_directory(dicom_dir, nifti_output_dir, compression=True)

# Buscar el nuevo NIfTI generado
nifti_files = [f for f in os.listdir(nifti_output_dir) if f.endswith(".nii.gz")]
if not nifti_files:
    raise FileNotFoundError("No se encontró ningún archivo .nii.gz")

nifti_path = os.path.join(nifti_output_dir, nifti_files[0])
print("✅ NIfTI generado:", nifti_path)

# ──────────────────────────────
# 4. CARGAR NIfTI Y CONVERTIR COORDENADAS
# ──────────────────────────────
img = nib.load(nifti_path)
data = img.get_fdata()
affine = img.affine
spacing = img.header.get_zooms()

# Convertir coordenadas físicas RAS → índice de vóxel
inv_affine = np.linalg.inv(affine)
center_voxel = nib.affines.apply_affine(inv_affine, center_ras)
center_voxel = np.round(center_voxel).astype(int)

# Radio en voxeles
radius_vox = np.round(radius_mm / np.array(spacing)).astype(int)

# ──────────────────────────────
# 5. MÁSCARA ESFÉRICA
# ──────────────────────────────
zz, yy, xx = np.ogrid[:data.shape[2], :data.shape[1], :data.shape[0]]
dist = ((xx - center_voxel[0])**2 / radius_vox[0]**2 +
        (yy - center_voxel[1])**2 / radius_vox[1]**2 +
        (zz - center_voxel[2])**2 / radius_vox[2]**2)
mask = dist <= 1.0
mask = mask.transpose(2, 1, 0)

subvolume = np.zeros_like(data)
subvolume[mask] = data[mask]

# ──────────────────────────────
# 6. GUARDADO NIfTI (RAS con header explícito)
# ──────────────────────────────
output_base = f"{folder_name}_E{int(radius_mm)}mm"
nii_path = os.path.join(nifti_output_dir, output_base + ".nii.gz")

nifti_img = nib.Nifti1Image(subvolume, affine)
nifti_img.header.set_sform(affine, code=1)
nifti_img.header.set_qform(affine, code=1)
nib.save(nifti_img, nii_path)
print(f"✅ Subvolumen guardado como NIfTI: {nii_path}")

# ──────────────────────────────
# 7. GUARDADO NRRD (LPS explícito)
# ──────────────────────────────
nrrd_path = os.path.join(nifti_output_dir, output_base + ".nrrd")

header = {
    'type': subvolume.dtype.name,
    'dimension': 3,
    'sizes': subvolume.shape,
    'space': 'left-posterior-superior',
    'space origin': affine[:3, 3].tolist(),
    'space directions': [affine[:3, i].tolist() for i in range(3)],
    'kinds': ['domain'] * 3,
    'endian': 'little',
    'encoding': 'gzip'
}

nrrd.write(nrrd_path, subvolume, header)
print(f"🟢 Subvolumen guardado como NRRD: {nrrd_path}")

# ──────────────────────────────
# 8. FIDUCIAL .FCSV PARA SLICER
# ──────────────────────────────
fiducial_path = os.path.join(nifti_output_dir, output_base + "_center.fcsv")
with open(fiducial_path, "w") as f:
    f.write("# Markups fiducial file version = 4.11\n")
    f.write("# CoordinateSystem = 0\n")  # 0 = RAS
    f.write("# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID\n")
    f.write(f"1,{center_ras[0]},{center_ras[1]},{center_ras[2]},0,0,0,1,1,1,0,CenterRAS,,\n")
print(f"📍 Fiducial guardado: {fiducial_path}")

# ──────────────────────────────
# 9. SEGMENTACIÓN .seg.nrrd
# ──────────────────────────────
seg_path = os.path.join(nifti_output_dir, output_base + ".seg.nrrd")

seg_header = {
    'type': 'uint8',
    'dimension': 3,
    'sizes': subvolume.shape,
    'space': 'left-posterior-superior',
    'space origin': affine[:3, 3].tolist(),
    'space directions': [affine[:3, i].tolist() for i in range(3)],
    'kinds': ['domain'] * 3,
    'endian': 'little',
    'encoding': 'gzip',
    'modality': 'SEG',
    'Segment0_Color': '1 0 0',  # rojo
    'Segment0_Name': 'Aneurisma',
    'Segment0_ID': 'aneurisma_seg',
    'Segment0_LabelValue': '1',
    'Segment0_Extent': f"0 {subvolume.shape[0]-1} 0 {subvolume.shape[1]-1} 0 {subvolume.shape[2]-1}",
    'Segment0_ReferenceImageExtentOffset': '0 0 0'
}

seg_data = (subvolume > 0).astype(np.uint8)
nrrd.write(seg_path, seg_data, seg_header)
print(f"🧠 Segmentación guardada como: {seg_path}")

# ──────────────────────────────
# 10. COPIA TODO A GOOGLE DRIVE
# ──────────────────────────────
shutil.copy(nii_path, os.path.join(base_dir, os.path.basename(nii_path)))
shutil.copy(nrrd_path, os.path.join(base_dir, os.path.basename(nrrd_path)))
shutil.copy(fiducial_path, os.path.join(base_dir, os.path.basename(fiducial_path)))
shutil.copy(seg_path, os.path.join(base_dir, os.path.basename(seg_path)))
print("📁✅ Todos los archivos copiados a Google Drive.")


📌 ¿Cómo usarla en Slicer?
Abrí la imagen base (.nii.gz o .nrrd).

Luego abrí el .seg.nrrd → Slicer lo cargará como una segmentación activa.

Activá la vista 3D o el Segment Editor para manipular la máscara.

[20250505 2:46 [ChatGPT]](https://chatgpt.com/share/681809f5-def4-8011-853b-5dd735a4b287)

